In [1]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.2

     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=54f61cb6d133e6c1b420a1a6fd2f6f5124a860e49dc53b474a7801623d02c790
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 2.5 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
ERROR: Could not find a version that satisfies the requirement torch==1.8.2 (from versions: 0.4.1, 0.4.1.post2, 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.

In [2]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-cdgd_6m7/kobert-tokenizer_2b88d9669b9d4fe4905e5809a98dca80
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-cdgd_6m7/kobert-tokenizer_2b88d9669b9d4fe4905e5809a98dca80
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4682 sha256=d30d1353a49e63558d6751c6f67d0fc43cbe05d6b6b38d091203c61aac6eecb5
  Stored in directory: /tmp/pip-ephem-wheel-cache-xwzhovpp/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model
# from kobert.pytorch_kobert import get_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
import sentencepiece
import transformers
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


# 에러나서 다른 방법 사용

In [4]:
print(torch.__version__) #1.8.1
print(transformers.__version__) #4.8.2

1.10.0+cu111
4.8.2


In [5]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-unctanx4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-unctanx4
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=43954afeef304095bab575319a8ceae444d432754a0c20ab4ece83483244c9bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-r7hrc86_/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [6]:
from kobert_tokenizer import KoBERTTokenizer
from kobert.pytorch_kobert import get_kobert_model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

In [7]:
#띄어쓰기에 강건한 모델
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

In [8]:
import torch
from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')
# text = "한국어 모델을 공유합니다."
# inputs = tokenizer.batch_encode_plus([text])
# out = model(input_ids = torch.tensor(inputs['input_ids']),attention_mask = torch.tensor(inputs['attention_mask']))
# out.pooler_output.shape

In [9]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# 데이터 전처리

In [10]:
all_saturi_label = pd.read_csv('/content/drive/MyDrive/자연어처리 과제/사투리_감정분류_데이터셋.csv', index_col = 0)

In [11]:
all_saturi_label

,label,saturi
0,0+Positive,그리 마스크 쫌 불편한 점도 있지만 좋은 점이 너무 많아.
1,3+Negative,드러누워 있으면 할아버지가 같이 식사를 못 하시는 거라.
2,3+Negative,그래 가지고 쫌 그런 어~ 아기자기하고 쪼끔 사람들이 덜 ...
3,0+Negative,백건우근데 안에 분위기 자체 거의 그냥 앵간한 모텔급으로 그냥 침대 있고 근데 ...
4,0+Negative,셋방이 미니 일 층이에요 그니까 이쪽에서 들어가면 일 층이고
...,...,...
49995,2+Positive,이*화 문화재청에 맡겨놔서 도난을 안 당했다고 내가 계속 말혔잖여.
49996,3+Negative,이게 사람마다 이게 크기도 다 다르고 이케 홀도 다 다른데
49997,1+Negative,벽지가 갑자기 새로 도배했는데 갑자기 다 이렇게 묽어져서 물이 새고
49998,0+Negative,동네 주민들이 거기 쫌 많이 가는 데 요즘에는 근데 쫌 ...


In [12]:
all_saturi_label['saturi'] = all_saturi_label['saturi'].str.replace('[a-z]+_[a-z]*','')
all_saturi_label['saturi'] = all_saturi_label['saturi'].str.strip()

In [13]:
all_saturi_label.loc[(all_saturi_label['label'] == "0+Positive"), 'label'] = 0  #기쁨+긍정 => 0
all_saturi_label.loc[(all_saturi_label['label'] == "0+Negative"), 'label'] = 1  #기쁨+부정 => 1
all_saturi_label.loc[(all_saturi_label['label'] == "1+Positive"), 'label'] = 2  #당황+긍정 => 2
all_saturi_label.loc[(all_saturi_label['label'] == "1+Negative"), 'label'] = 3  #당황+부정 => 3
all_saturi_label.loc[(all_saturi_label['label'] == "2+Positive"), 'label'] = 4  #분노+긍정 => 4
all_saturi_label.loc[(all_saturi_label['label'] == "2+Negative"), 'label'] = 5  #분노+부정 => 5
all_saturi_label.loc[(all_saturi_label['label'] == "3+Positive"), 'label'] = 6  #불안+긍정 => 6
all_saturi_label.loc[(all_saturi_label['label'] == "3+Negative"), 'label'] = 7  #불안+부정 => 7
all_saturi_label.loc[(all_saturi_label['label'] == "4+Positive"), 'label'] = 8  #상처+긍정 => 8
all_saturi_label.loc[(all_saturi_label['label'] == "4+Negative"), 'label'] = 9  #상처+부정 => 9
all_saturi_label.loc[(all_saturi_label['label'] == "5+Positive"), 'label'] = 10  #슬픔+긍정 => 10
all_saturi_label.loc[(all_saturi_label['label'] == "5+Negative"), 'label'] = 11  #슬픔+부정 => 11

data_list = []
for q, label in zip(all_saturi_label['saturi'], all_saturi_label['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [14]:
data_list

[['그리 마스크 쫌   불편한 점도 있지만 좋은 점이 너무 많아.', '0'],
 ['드러누워 있으면 할아버지가 같이 식사를 못 하시는 거라.', '7'],
 ['그래 가지고 쫌   그런 어~ 아기자기하고 쪼끔 사람들이 덜 붐비는?', '7'],
 ['백건우근데 안에 분위기 자체 거의 그냥 앵간한   모텔급으로 그냥 침대 있고 근데 전기장판만 없는 거지 침대에', '1'],
 ['셋방이 미니 일 층이에요 그니까   이쪽에서 들어가면 일 층이고', '1'],
 ['지가   꼭 그렇게 하고 다니고 아무리 막 임신을 하고 다녀도 지   스타일 딱 지키더라고 사무실을 갈 때도', '5'],
 ['성격은 이게 하고 싶어도 참는 거야 내가 내가 잠 잘 자고 내가 편할라고', '5'],
 ['xxxx 큰아이 족은아이  가 지금 인천에 고치   사는디', '3'],
 ['한 오천만 원 정도 했던 것 같애   오천만 원? 오천도 뭐', '9'],
 ['야가 얘가 아이고   자가   자고    억수로   막 헷갈리게', '7'],
 ['그러니까 나는 근데 실제 성격이랑 다른 거에서 진짜 연기면 연기 잘한다고 느껸.', '2'],
 ['그래 눈으   바쩍   갖다 대고 조준궁기  를 들여다 봐야지.', '7'],
 ['걱정이야 괜히  걱정하고 걱정만 하고 있어서 잘 하지도 못하면서', '3'],
 ['왜? 그냥 사람인 게 아 맞아. 이   그러면 사람이면 언제 죽을지 모르지?', '7'],
 ['나는 초반에 경   못해났거든 그래서 막 검색해 보고 경해나신디', '3'],
 ['이예진나는 막~ 대전 밖을 많이 벗어나는 편이 아니여서', '7'],
 ['굉장히 괴롭고 막 니도   음악 좋아하잖아 어~ 음악 좋아하니까 그~ 악기에도 관심이 많고 막 그렇다   아이가   근데', '9'],
 ['네 이러고 있는데 수화기 들어가지고 경찰을 부를려고   대기를 타고 있었어.', '7'],
 ['뭐~ 그것도  이 이제 뭐~ 원래가 차이가 기계마다 차이가 나니까네', '11'],
 ['약간

In [15]:
# import random

# data_list = random.sample(data_list, 50000)

In [16]:
# import pickle

# with open("/content/drive/MyDrive/자연어처리 과제/model/data_list_1125.txt", "rb") as fp:   # Unpickling
#     data_list = pickle.load(fp)

In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [18]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [19]:
print(len(dataset_train))
print(len(dataset_test))

40000
10000


In [20]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['그리 마스크 쫌   불편한 점도 있지만 좋은 점이 너무 많아.', '0']
['뭐~ 기생수 도쿄 구울이랑 유명한 것들 있잖아요 그런 것도 다 재밌게 본 거 같아요.', '0']
['야 우리 고등학교 이 학년 때 너네 너랑&name2& 랑 &name4&공부하켄   해가지고 시험 때 공부하고 있으면', '9']
['계획을 세우 것도 쫌   어려워해요 영어 여행을 가더라도 그냥 쫌   이렇게', '7']
['쫌   빨리 준비하는 편인가요 엔씨엔스는 거의 대부분 자격증 같은 거 다 준비하고 나서 인제', '7']
['하*꼼   영   영하*꼼좀 영   색다르게시리   하*꼼   들영있고', '4']
['나도 인제 직장을 다니다가 그만두고 새로운 도전을 하잖아.', '3']


In [21]:
# Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화 및 모델 돌리기

In [22]:
#토큰화
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=12,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
device = torch.device("cpu")

#BERT 모델 불러오기
# model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model = torch.load('/content/drive/MyDrive/자연어처리 과제/model/사투리_감정분류_라벨12개_epoch5.h5')

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 모델 학습

In [ ]:
#이건 표준어 감정분류 모델 학습한 것

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.1458371877670288 train acc 0.5390625
epoch 1 train acc 0.6169386942675159


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 3 test acc 0.6388671875


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.8615317344665527 train acc 0.69140625
epoch 4 train acc 0.7614201831210191


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 4 test acc 0.6376953125


In [90]:
#이건 사투리 감정분류 모델 학습한 것

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.5046443939208984 train acc 0.140625
epoch 1 batch id 201 loss 1.9366549253463745 train acc 0.23503575870646767
epoch 1 train acc 0.28354632587859424


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.47151898734177217


  0%|          | 0/313 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.9230576753616333 train acc 0.703125
epoch 5 batch id 201 loss 0.790044903755188 train acc 0.732548196517413
epoch 5 train acc 0.7400409345047924


  0%|          | 0/79 [00:00<?, ?it/s]

epoch 5 test acc 0.4775514240506329


In [ ]:
#이건 사투리 감정분류 모델 학습한 것(epoch 5~10)

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.8856605291366577 train acc 0.703125
epoch 1 batch id 201 loss 1.0198101997375488 train acc 0.6856343283582089
epoch 1 train acc 0.6721994808306709


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 batch id 1 loss 0.6748337745666504 train acc 0.78125
epoch 4 batch id 201 loss 0.3397710919380188 train acc 0.8453047263681592
epoch 4 train acc 0.8565545127795527


  0%|          | 0/79 [00:00<?, ?it/s]

epoch 4 test acc 0.47151898734177217


  0%|          | 0/313 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3623434603214264 train acc 0.9140625
epoch 5 batch id 201 loss 0.2602090537548065 train acc 0.8967661691542289


In [ ]:
torch.save(model, '/content/drive/MyDrive/자연어처리 과제/model/사투리_감정분류_라벨12개_epoch10.h5')

# 검증

In [ ]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    saved_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = saved_model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("상처가")
            elif np.argmax(logits) == 5:
                test_eval.append("슬픔이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 어머나


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 어머나 시발 저게뭐야
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 죽여버리기 전에 조용히해
>> 입력하신 내용에서 불안이 느껴집니다.




KeyboardInterrupt: ignored

In [ ]:
""